In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install parlai
!parlai display_data -t msc:PersonaSummary --include-last-session True 
!pip install names
!pip install gingerit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.3 MB/s eta 0:00

2023-06-11 11:20:13.768673: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 11:20:14.664590: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11:20:17 | Opt:
11:20:17 |     allow_missing_init_opts: False
11:20:17 |     batchsize: 1
11:20:17 |     datapath: /usr/local/lib/python3.10/dist-packages/data
11:20:17 |     datatype: train:ordered
11:20:17 |     dict_class: None
11:20:17 |     display_add_fields: 
11:20:17 |     download_path: None
11:20:17 |     dynamic_batching: None
11:20:17 |     hide_labels: False
11:20:17 |     ignore_agent_reply: True
11:20:17 |     image_cropsize: 224
11:20:17 |     image_mode: raw
11:20:17 |     image_size: 256
11:20:17 |     include_last_session: True
11:20:17 | 

In [3]:
import re
import json
# Read the JSON file
folder_path = '/usr/local/lib/python3.10/dist-packages/data/msc/msc/msc_personasummary'

# Data PreProcess Function
def createDataset(mode): 
    s1_data_path = f"{folder_path}/session_1/{mode}.txt"
    s2_data_path = f"{folder_path}/session_2/{mode}.txt"
    s3_data_path = f"{folder_path}/session_3/{mode}.txt"
    s4_data_path = f"{folder_path}/session_4/{mode}.txt"

    if mode == 'train':
        data_paths = [s1_data_path, s2_data_path, s3_data_path]
    else:
        data_paths = [s1_data_path, s2_data_path, s3_data_path, s4_data_path]

    mode_X = []
    mode_y = []
    # 데이터 파일 열기

    for data_path in data_paths:
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                # 한 줄씩 읽기
                data = json.loads(line)
                id0_prev_count = 0
                id1_prev_count = 0
                # print(data['dialog'])
                for entry in data['dialog']:
                    bot_id = entry['id']
                    agg_persona_list = entry['agg_persona_list']
                    if bot_id == 'bot_0':
                        cur_agg_persona_list = agg_persona_list[id0_prev_count:]
                    else:
                        cur_agg_persona_list = agg_persona_list[id1_prev_count:]
                    text = entry['text']
                    summary = " ".join(cur_agg_persona_list)
                    if bot_id == 'bot_0':
                        id0_prev_count += len(cur_agg_persona_list)
                    else:
                        id1_prev_count += len(cur_agg_persona_list)

                    mode_X.append(text)
                    mode_y.append(summary)

    return mode_X, mode_y

In [4]:
train_X, train_y = createDataset('train')
msc_train_dataset = {'text' : train_X, 'summary' : train_y}
valid_X, valid_y = createDataset('valid')
msc_valid_dataset = {'text': valid_X, 'summary' : valid_y}
test_X, test_y = createDataset('test')
msc_test_dataset = {'text' : test_X, 'summary' : test_y}

In [5]:
final_dataset = {'train' : [], 'valid' : [], 'test' : []}

def add_dataset(dataset, mode):
    textDataset = dataset['text']
    summDataset = dataset['summary']
    for sentence, summary in zip(textDataset, summDataset):
        line = '<Text> '+ sentence + ' <Summary> ' + summary +'\n'
        final_dataset[mode].append(line)

In [6]:
add_dataset(msc_train_dataset, 'train')
add_dataset(msc_valid_dataset, 'valid')
add_dataset(msc_test_dataset, 'test')

In [7]:
def view_lines(lst, length=20):
    for i in range(length):
        print(lst[i])

In [ ]:
view_lines(final_dataset['train'])

In [9]:
import random

for ds in final_dataset:
    random.shuffle(final_dataset[ds])

In [12]:
len(final_dataset['train']), len(final_dataset['valid']), len(final_dataset['test'])

(133290, 25459, 24375)

In [17]:
def countSummaryType(dataset):
    count_empty = 0
    for summary in dataset['summary']:
        if summary == '':
            count_empty += 1
    return len(dataset['summary']) - count_empty, count_empty

In [16]:
countSummaryType(msc_train_dataset)

(83575, 49715)

In [23]:
def viewSummary(dataset, length=10):
    j = 0
    for i, summary in enumerate(dataset['summary']):
        if summary != '':
            print('<Text> ', dataset['text'][i], ' <Summary> ', dataset['summary'][i])
            j += 1
        if j == length:
            break

In [24]:
viewSummary(msc_train_dataset)

<Text>  I have been all over the world. I'm military.  <Summary>  I served or serve in the military. I've traveled the world.
<Text>  Sure do. And a lot of experience blowing things up! Haha. Bora bora is nice.  <Summary>  I've blown things up.
<Text>  I've been working non stop crazy hours and need a break.  <Summary>  I've been working a lot of extra hours. I want to break from my non-stop work.
<Text>  Nope... Just sounds nice, and repetitive. Bora... Bora. Ha!  <Summary>  I've never been to Bora Bora.
<Text>  Kittens really? I rather be at the beach.  <Summary>  I like going to the beach.
<Text>  Or maybe brownies... I love chocolate.  <Summary>  I love chocolate.
<Text>  I love brownies too but I haven't quite perfected mine yet.  <Summary>  I love brownies.
<Text>  I'm reading for a exam I've and you?  <Summary>  I have an exam soon.
<Text>  Just finished playing with my three dogs here at home.  <Summary>  I have three dogs.
<Text>  Cool 3 dogs. I'll get a dog when I'm done with

In [25]:
def reduceEmptySummary(dataset, tolerable_empty_count=10000):
    processed_dataset = {'text':[], 'summary':[]}
    empty_count = 0
    for i, summary in enumerate(dataset['summary']):
        if summary != '':
            processed_dataset['text'].append(dataset['text'][i])
            processed_dataset['summary'].append(dataset['summary'][i])
        else:
            if empty_count < tolerable_empty_count:
                processed_dataset['text'].append(dataset['text'][i])
                processed_dataset['summary'].append('')
                empty_count += 1
    return processed_dataset

In [26]:
reduced_msc_train_dataset = reduceEmptySummary(msc_train_dataset)

In [27]:
countSummaryType(reduced_msc_train_dataset)

(83575, 10000)

In [29]:
final_dataset['train'] = []
add_dataset(reduced_msc_train_dataset, 'train')

In [30]:
len(final_dataset['train']), len(final_dataset['valid']), len(final_dataset['test'])

(93575, 25459, 24375)

In [31]:
for ds in final_dataset:
    random.shuffle(final_dataset[ds])

In [33]:
def save_dataset(lst, file_path):
    with open(file_path, "w") as file:
        for line in lst:
            file.write(line + "\n")

In [34]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/Metabuddy/Data Process/week2'
file_path = f'{folder_path}/train.txt'
save_dataset(final_dataset['train'], file_path)
file_path = f'{folder_path}/valid.txt'
save_dataset(final_dataset['valid'], file_path)
file_path = f'{folder_path}/test.txt'
save_dataset(final_dataset['test'], file_path)